In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [2]:
class RegressionModel(nn.Module):
    def __init__(self, indim, outdim):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 1, kernel_size=3, stride=1, padding=1)
        self.linear = nn.Linear(indim, outdim)

    def forward(self, x):
        y = self.conv1(x)
        y = self.linear(y)
        print('fwd:', y.size())
        return y

regression_model = RegressionModel(10, 5)

In [3]:
print(regression_model.linear.weight.size(), regression_model.linear.bias.size())

torch.Size([5, 10]) torch.Size([5])


In [22]:
N = 500  # size of toy data
indim = 10
outdim = 5

def build_linear_dataset(N, indim, outdim, noise_std=0.01):
    X = np.random.rand(N, indim)
    # w = 3
    w = 3 * np.ones((indim, outdim))
    # b = 1
    b = 1*np.ones((N, outdim))
    y = np.matmul(X, w)
#     print(y.shape)
    y = y + b + np.random.normal(0, noise_std, size=(N, outdim))
#     print(y.shape)
    y = y.reshape(N, outdim)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    return data

In [ ]:
build_linear_dataset(N, indim, outdim).size()

In [25]:
def model(data):
    # Create unit normal priors over the parameters
    loc, scale = torch.zeros(outdim, indim), 10 * torch.ones(outdim, indim)
    bias_loc, bias_scale = torch.zeros(outdim), 10 * torch.ones(outdim)
    w_prior = Normal(loc, scale).independent(1)
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    #how does sampled prior look?
    print('how does sampled prior look?')
    print('w_prior:', w_prior.sample().size())
    print('b_prior:', b_prior.sample().size())
    
    
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    with pyro.iarange("map", N):
        x_data = data[:, :-5].view(N,1,-1)
        y_data = data[:, -5:].view(N, 1, -1)
        # run the regressor forward conditioned on data
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        print('pred_mean shape', prediction_mean.size())
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1*torch.ones(data.size(0), 1, outdim)).independent(1),
                    obs=y_data)

In [27]:
softplus = torch.nn.Softplus()

def guide(data):
    # define our variational parameters
    w_loc = torch.randn(outdim, indim)
    # note that we initialize our scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(outdim, indim) + 0.05 * torch.randn(outdim, indim))
    b_loc = torch.randn(outdim)
    b_log_sig = torch.tensor(-3.0 * torch.ones(outdim) + 0.05 * torch.randn(outdim))
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [28]:
num_iterations = 1000 if not smoke_test else 2
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N, 10, 5)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))

if __name__ == '__main__':
    main()

how does sampled prior look?
w_prior: torch.Size([5, 10])
b_prior: torch.Size([5])
fwd: torch.Size([500, 1, 5])
pred_mean shape torch.Size([500, 1, 5])


ValueError: at site "module$$$linear.weight", invalid log_prob shape
  Expected [], actual [5]
  Try one of the following fixes:
  - enclose the batched tensor in a with iarange(...): context
  - .independent(...) the distribution being sampled
  - .permute() data dimensions